# Data Preprocessing

### Time Aware LOO Split

In [3]:
import pandas as pd
from pathlib import Path

RAW = Path(r"C:\Users\abdul\ece1508gp\movielens_dataset")
out_csv = RAW / "ratings.csv"

df = pd.read_csv(
    RAW / "ratings.dat",
    sep="::",
    engine="python",
    names=["userId", "movieId", "rating", "timestamp"]
)
df.to_csv(out_csv, index=False)
print("wrote:", out_csv, len(df))

wrote: C:\Users\abdul\ece1508gp\movielens_dataset\ratings.csv 1000209


In [5]:
# Time-aware Leave-One-Out (LOO) split for MovieLens ratings
import pandas as pd
import numpy as np
from pathlib import Path

def _detect_ts_unit(ts_series):
    vmax = float(ts_series.max())
    return "ms" if vmax > 1e12 else "s"

def time_aware_loo_split(
    ratings_csv: str,
    out_dir: str,
    rating_threshold: float = 3.0,
    min_positives: int = 3,
    also_csv: bool = False,
):
    out = Path(out_dir); (out / "splits").mkdir(parents=True, exist_ok=True)
    ratings = pd.read_csv(ratings_csv)

    # 1) Basic checks
    need = {"userId","movieId","rating","timestamp"}
    missing = need - set(ratings.columns)
    if missing:
        raise ValueError(f"ratings.csv missing columns: {missing}")

    # 2) Normalize timestamp and keep implicit positives
    unit = _detect_ts_unit(ratings["timestamp"])
    ratings["ts"] = pd.to_datetime(ratings["timestamp"], unit=unit)

    pos = ratings[ratings["rating"] >= rating_threshold].copy()
    # Drop duplicate (user,item) -> keep earliest
    pos = pos.sort_values(["userId","ts","movieId"], kind="mergesort")
    pos = pos.drop_duplicates(["userId","movieId"], keep="first")

    # Keep users with at least min_positives
    pos = pos[pos.groupby("userId")["movieId"].transform("size") >= min_positives].copy()

    # 3) Rank by time per user, assign splits: last=test, second last=val (if >=3), rest=train
    pos["n"] = pos.groupby("userId")["movieId"].transform("size")
    pos["idx"] = pos.groupby("userId").cumcount()
    pos["split"] = "train"
    pos.loc[pos["idx"] == pos["n"]-1, "split"] = "test"
    pos.loc[(pos["n"]>=3) & (pos["idx"] == pos["n"]-2), "split"] = "val"

    train = pos[pos["split"]=="train"][["userId","movieId","ts"]].reset_index(drop=True)
    val_targets  = pos[pos["split"]=="val"][["userId","movieId","ts"]].rename(
        columns={"movieId":"val_item","ts":"ts_val"}).reset_index(drop=True)
    test_targets = pos[pos["split"]=="test"][["userId","movieId","ts"]].rename(
        columns={"movieId":"test_item","ts":"ts_test"}).reset_index(drop=True)

    # 4) Build ID maps from TRAIN only (contiguous 0..U-1 and 0..I-1)
    uids = pd.DataFrame(sorted(train["userId"].unique()), columns=["userId"])
    uids["uid"] = range(len(uids))
    iids = pd.DataFrame(sorted(train["movieId"].unique()), columns=["movieId"])
    iids["iid"] = range(len(iids))

    # 5) Also provide indexed versions (useful for MF/ALS)
    train_idx = (train.merge(uids, on="userId", how="inner")
                      .merge(iids, on="movieId", how="inner"))
    val_idx = None
    if len(val_targets):
        val_idx = (val_targets.merge(uids, on="userId", how="inner")
                              .merge(iids, left_on="val_item", right_on="movieId", how="left")
                              .drop(columns=["movieId"]))
    test_idx = (test_targets.merge(uids, on="userId", how="inner")
                               .merge(iids, left_on="test_item", right_on="movieId", how="left")
                               .drop(columns=["movieId"]))

    # 6) Save outputs
    sp = out / "splits"
    train.to_parquet(sp / "train.parquet", index=False)
    if len(val_targets):
        val_targets.to_parquet(sp / "val_targets.parquet", index=False)
    test_targets.to_parquet(sp / "test_targets.parquet", index=False)
    uids.to_parquet(sp / "user_id_map.parquet", index=False)
    iids.to_parquet(sp / "item_id_map.parquet", index=False)
    train_idx.to_parquet(sp / "train_indexed.parquet", index=False)
    if val_idx is not None:
        val_idx.to_parquet(sp / "val_targets_indexed.parquet", index=False)
    test_idx.to_parquet(sp / "test_targets_indexed.parquet", index=False)

    if also_csv:
        train.to_csv(sp / "train.csv", index=False)
        if len(val_targets): val_targets.to_csv(sp / "val_targets.csv", index=False)
        test_targets.to_csv(sp / "test_targets.csv", index=False)
        uids.to_csv(sp / "user_id_map.csv", index=False)
        iids.to_csv(sp / "item_id_map.csv", index=False)
        train_idx.to_csv(sp / "train_indexed.csv", index=False)
        if val_idx is not None:
            val_idx.to_csv(sp / "val_targets_indexed.csv", index=False)
        test_idx.to_csv(sp / "test_targets_indexed.csv", index=False)

    # 7) Quick stats + cold-start counts
    cold_val = int(val_idx["iid"].isna().sum()) if val_idx is not None and "iid" in val_idx else 0
    cold_test = int(test_idx["iid"].isna().sum()) if "iid" in test_idx else 0
    stats = f"""Time-aware LOO split summary
Users (TRAIN map): {len(uids)}
Items (TRAIN map): {len(iids)}
TRAIN positives  : {len(train)}
VAL users        : {len(val_targets["userId"].unique()) if len(val_targets) else 0}
TEST users       : {len(test_targets["userId"].unique())}
Cold-start VAL items  : {cold_val}
Cold-start TEST items : {cold_test}
"""
    (sp / "stats.txt").write_text(stats, encoding="utf-8")
    print(stats)


### Cap it to only 5 movies per user

In [9]:
import pandas as pd

# Load ratings
ratings = pd.read_csv(r"C:\Users\abdul\ece1508gp\movielens_dataset\ratings.csv")

# Sort by userId and timestamp (most recent first)
ratings = ratings.sort_values(["userId", "timestamp"], ascending=[True, False])

# Keep only the 5 most recent ratings per user
ratings_limited = ratings.groupby("userId").head(5).reset_index(drop=True)

print("Limited ratings shape:", ratings_limited.shape)

# Optionally, save for downstream steps
ratings_limited.to_csv(r"C:\Users\abdul\ece1508gp\movielens_dataset\ratings_limited.csv", index=False)

Limited ratings shape: (30200, 4)


### Call Time Aware LOO Split

In [10]:
time_aware_loo_split(
    ratings_csv=r"C:\Users\abdul\ece1508gp\movielens_dataset\ratings_limited.csv",
    out_dir=r"C:\Users\abdul\ece1508gp\movielens_dataset",
    rating_threshold=3.0,   # implicit “like” threshold
    min_positives=3,        # drop users with <3 positives
    also_csv=True           # optional CSV mirrors
)

Time-aware LOO split summary
Users (TRAIN map): 5413
Items (TRAIN map): 2287
TRAIN positives  : 13295
VAL users        : 5413
TEST users       : 5413
Cold-start VAL items  : 306
Cold-start TEST items : 343



# Load splits + quick helpers

In [11]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.sparse import csr_matrix

SPLITS = Path(r"C:\Users\abdul\ece1508gp\movielens_dataset\splits")

train = pd.read_parquet(SPLITS / "train_indexed.parquet")         # [uid, iid, ts]
val_idx = pd.read_parquet(SPLITS / "val_targets_indexed.parquet") # [userId, uid, val_item, iid, ts_val]
test_idx = pd.read_parquet(SPLITS / "test_targets_indexed.parquet")  # [userId, uid, test_item, iid, ts_test]

# sizes
U = int(train["uid"].max()) + 1
I = int(train["iid"].max()) + 1
print("users:", U, "items:", I, "train rows:", len(train))

# users×items implicit matrix (Windows-friendly dtypes)
rows = train["uid"].to_numpy(dtype=np.int32, copy=False)
cols = train["iid"].to_numpy(dtype=np.int32, copy=False)
data = np.ones(len(train), dtype=np.float32)

R = csr_matrix((data, (rows, cols)), shape=(U, I), dtype=np.float32)

# fast lookup: items seen in TRAIN per user
user_seen = train.groupby("uid")["iid"].apply(set).to_dict()

# evaluation helper
def candidate_coverage(cand_df, targets_df, tgt_col="iid"):
    df = cand_df.merge(targets_df[["uid", tgt_col]], on="uid", how="inner")
    df = df[df[tgt_col].notna()]  # drop cold-start items
    return np.mean([int(t) in set(c) for t, c in zip(df[tgt_col], df["candidates"])])


users: 5413 items: 2287 train rows: 13295


# Artifact A: Popularity (train-only)

In [12]:
# TRAIN-only popularity
pop = (
    train.groupby("iid").size()
         .sort_values(ascending=False)
)

def top_pop_unseen(u_seen, P=50):
    out = []
    for iid in pop.index:
        if iid not in u_seen:
            out.append(int(iid))
            if len(out) >= P:
                break
    return out


# Artifact B: item-item (w sklearn)

In [13]:
from sklearn.neighbors import NearestNeighbors

# item × user matrix
item_users = R.T.tocsr()
item_users.sort_indices()

knn = NearestNeighbors(
    n_neighbors=51,      # 50 real neighbors
    metric="cosine",
    algorithm="brute",
    n_jobs=-1,
)
knn.fit(item_users)
print("sklearn item–item fitted on", item_users.shape)

def item_neighbors_from_history_sklearn(u_seen, per_item=20):
    C = []
    for iid in u_seen:
        dists, idxs = knn.kneighbors(item_users[iid], n_neighbors=per_item+1, return_distance=True)
        neigh_ids = idxs[0][1:]   # drop self
        C.extend(int(j) for j in neigh_ids)
    return C


sklearn item–item fitted on (2287, 5413)


# Artifact C: ALS Train only

In [14]:
from implicit.als import AlternatingLeastSquares

# ALS model (we can tune later)
als = AlternatingLeastSquares(
    factors=64,
    regularization=0.05,
    iterations=10,
    use_gpu=False
)

# implicit ALS wants item×user
als.fit(R.T.tocsr())

print("ALS trained.")


c:\Users\abdul\.venvs\ece1508gp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\abdul\.venvs\ece1508gp\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 10/10 [00:00<00:00, 57.07it/s]

ALS trained.


# Build candidate pools (w artifacts - popularity/item-item,als)

In [15]:
from tqdm import tqdm
import pyarrow as pa, pyarrow.parquet as pq

def mf_top_unseen(uid: int, Rk):
    # ALS recommend already filters liked items
    ids, scores = als.recommend(
        userid=int(uid),
        user_items=R[int(uid)],
        N=Rk,
        filter_already_liked_items=True,
        recalculate_user=True,
    )
    return [int(i) for i in ids]

def build_pool_for_user(uid: int,
                        P=200,        # how many pop to try
                        K=400,       # final cap
                        Rk=200,       # how many ALS recs to try
                        knn_per_item=20):  # how many neighbors per seen item
    seen = user_seen.get(uid, set())
    seen_set = set(seen)

    C = []

    # 1) popularity (train-only)
    C += top_pop_unseen(seen, P=P)

    # 2) ALS / MF (only if user is warm)
    if len(seen):
        C += mf_top_unseen(uid, Rk=Rk)

    # 3) item–item (sklearn) — only if user has history
    if len(seen):
        few = list(seen)[:12]          # only 5 items from history
        C += item_neighbors_from_history_sklearn(few, per_item=knn_per_item)

    # 4) de-dup + drop seen + cap
    dedup = []
    for iid in C:
        if iid in seen_set:
            continue
        if iid in dedup:
            continue
        dedup.append(iid)
        if len(dedup) >= K:
            break

    return dedup


def make_candidates(user_ids, P=200, K=400, Rk=200, knn_per_item=20):
    rows = []
    for u in tqdm(user_ids):
        rows.append({
            "uid": int(u),
            "candidates": build_pool_for_user(
                int(u),
                P=P,
                K=K,
                Rk=Rk,
                knn_per_item=knn_per_item,
            )
        })
    return pd.DataFrame(rows)


# who to build for
users_val  = sorted(val_idx["uid"].unique())  if len(val_idx)  else []
users_test = sorted(test_idx["uid"].unique()) if len(test_idx) else []

# build pools
cand_val  = make_candidates(users_val,  P=20,  K=50, Rk=20, knn_per_item=10)
cand_test = make_candidates(users_test, P=20,  K=50, Rk=20, knn_per_item=10)

# save
OUT = SPLITS.parent / "candidates"
OUT.mkdir(exist_ok=True)
cand_val.to_parquet(OUT / "val.parquet", index=False)
cand_test.to_parquet(OUT / "test.parquet", index=False)
print("Wrote candidates to:", OUT)


100%|██████████| 5413/5413 [11:14<00:00,  8.03it/s]


Wrote candidates to: C:\Users\abdul\ece1508gp\movielens_dataset\candidates


# Check candidate coverage

In [16]:
val_cov  = candidate_coverage(cand_val,  val_idx,  "iid") if len(cand_val)  else float("nan")
test_cov = candidate_coverage(cand_test, test_idx, "iid") if len(cand_test) else float("nan")
print(f"Candidate coverage  val={val_cov:.2%}  test={test_cov:.2%}")

Candidate coverage  val=10.48%  test=9.53%


# Load & Define Covered Users

In [17]:
from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path("C:/Users/abdul/ece1508gp/movielens_dataset")
SPLITS = ROOT / "splits"
CANDS  = ROOT / "candidates"

train_idx = pd.read_parquet(SPLITS / "train_indexed.parquet")   # [uid,iid,ts]
val_idx   = pd.read_parquet(SPLITS / "val_targets_indexed.parquet")
test_idx  = pd.read_parquet(SPLITS / "test_targets_indexed.parquet")

cand_val  = pd.read_parquet(CANDS / "val.parquet")   # [uid, candidates(list[iid])]
cand_test = pd.read_parquet(CANDS / "test.parquet")

def coverage_rows(cands_df, targets_df, tgt_name):
    # merge candidates with target iid (indexed) for the same uid
    df = cands_df.merge(
        targets_df[["uid", "iid"]].rename(columns={"iid": tgt_name}),
        on="uid", how="inner"
    )

    # drop cold-start targets (iid is NaN) and rows without candidates
    before = len(df)
    df = df[df[tgt_name].notna() & df["candidates"].notna()].copy()
    dropped = before - len(df)

    # ensure candidates are lists
    df["candidates"] = df["candidates"].apply(lambda x: list(x) if isinstance(x, (list, tuple, np.ndarray)) else [])

    # use pandas nullable Int64 to avoid int(NaN) issues
    df[tgt_name] = df[tgt_name].astype("Int64")

    # membership test (no int(...) cast needed now)
    df["target_in_pool"] = [t in set(c) for t, c in zip(df[tgt_name], df["candidates"])]

    print(f"{tgt_name}: dropped {dropped} rows (cold-start/invalid); kept {len(df)}")
    return df

val_cov  = coverage_rows(cand_val,  val_idx,  "val_item")
test_cov = coverage_rows(cand_test, test_idx, "test_item")

val_covered  = val_cov[val_cov["target_in_pool"]].copy()
test_covered = test_cov[test_cov["target_in_pool"]].copy()

print(f"Coverage  val={len(val_covered)/len(val_cov):.2%}  test={len(test_covered)/len(test_cov):.2%}")


val_item: dropped 306 rows (cold-start/invalid); kept 5107
test_item: dropped 343 rows (cold-start/invalid); kept 5070
Coverage  val=10.48%  test=9.53%


# Get Covered Users

In [18]:
def mark_coverage(cands_df: pd.DataFrame, targets_df: pd.DataFrame, tgt_name: str):
    df = cands_df.merge(
        targets_df[["uid","iid"]].rename(columns={"iid": tgt_name}),
        on="uid", how="inner"
    )
    # guard against NaNs and non-lists
    df["candidates"] = df["candidates"].apply(
        lambda x: list(x) if isinstance(x, (list, tuple, np.ndarray, pd.Series)) else []
    )
    # fill NaNs to avoid int() on NaN
    tgt = df[tgt_name].fillna(-1).astype(int).to_numpy()
    df["target_in_pool"] = [int(t) in set(c) for t, c in zip(tgt, df["candidates"])]
    return df[["uid","target_in_pool"]]

val_mark  = mark_coverage(cand_val,  val_idx,  "val_item")
test_mark = mark_coverage(cand_test, test_idx, "test_item")

covered_val_uids  = set(val_mark.loc[val_mark["target_in_pool"], "uid"])
covered_test_uids = set(test_mark.loc[test_mark["target_in_pool"], "uid"])

print(f"Users valid for eval  val={len(covered_val_uids)}  test={len(covered_test_uids)}")

Users valid for eval  val=535  test=483


# Sanity check coverage

In [19]:
# Restrict targets and pools to the covered users only (these will have 100% coverage by construction)
val_eval_targets  = val_idx[val_idx["uid"].isin(covered_val_uids)].copy()
test_eval_targets = test_idx[test_idx["uid"].isin(covered_test_uids)].copy()

cand_val_eval  = cand_val[cand_val["uid"].isin(covered_val_uids)].reset_index(drop=True)
cand_test_eval = cand_test[cand_test["uid"].isin(covered_test_uids)].reset_index(drop=True)

# (Optional) sanity check – should print 100%/100%
def coverage_percent(cands_df, targets_df, tgt_col):
    df = cands_df.merge(targets_df[["uid", tgt_col]], on="uid")
    return np.mean([int(t) in set(c) for t, c in zip(df[tgt_col], df["candidates"])])

print("Eval coverage",
      f"val={coverage_percent(cand_val_eval,  val_eval_targets,  'iid'):.2%}",
      f"test={coverage_percent(cand_test_eval, test_eval_targets, 'iid'):.2%}")


Eval coverage val=100.00% test=100.00%


# Train ALS

In [20]:
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight 

# Build users × items CSR from train_idx 
U = int(train_idx["uid"].max()) + 1
I = int(train_idx["iid"].max()) + 1

rows = train_idx["uid"].to_numpy(dtype=np.int32, copy=False)
cols = train_idx["iid"].to_numpy(dtype=np.int32, copy=False)
data = np.ones(len(train_idx), dtype=np.float32)

alpha = 15
R = csr_matrix((data, (rows, cols)), shape=(U, I), dtype=np.float32)
X = bm25_weight(R, K1=100, B=0.8).astype(np.float32)

# Train ALS on users × items (keeps names aligned) 
als = AlternatingLeastSquares(
    factors=96,
    regularization=0.05,
    iterations=200,
    use_gpu=False,     
    dtype=np.float32,
)
als.fit(alpha*X)

# Cache factors with consistent shapes
U_f = als.user_factors   # shape [U, F]  user embeddings
V_f = als.item_factors   # shape [I, F]  item embeddings

print("ALS shapes  user_factors:", U_f.shape, " item_factors:", V_f.shape, " U:", U, " I:", I)
assert U_f.shape[0] == U and V_f.shape[0] == I


c:\Users\abdul\.venvs\ece1508gp\lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(
100%|██████████| 200/200 [00:04<00:00, 42.20it/s]

ALS shapes  user_factors: (5413, 96)  item_factors: (2287, 96)  U: 5413  I: 2287


# Eval

In [22]:
# 0) Join pools with their (covered) targets -> one tidy frame per split
val_eval  = (cand_val_eval
             .merge(val_eval_targets[["uid","iid"]]
                    .rename(columns={"iid":"target"}), on="uid"))
test_eval = (cand_test_eval
             .merge(test_eval_targets[["uid","iid"]]
                    .rename(columns={"iid":"target"}), on="uid"))

# 1) Cache ALS factors
U_f = als.user_factors            # [U, F]
V_f = als.item_factors            # [I, F]
I   = V_f.shape[0]

def score_als(uid: int, cand_iids: list[int]) -> np.ndarray:
    """Return ALS scores for the candidate items of a user."""
    if not cand_iids:
        return np.asarray([])
    u = U_f[int(uid)]
    # keep only in-bounds item ids (defensive)
    c = np.array([int(i) for i in cand_iids if 0 <= int(i) < I], dtype=np.int64)
    if c.size == 0:
        return np.asarray([])
    return V_f[c] @ u

def hit_at_k(ranked, tgt, k=10):
    return 1.0 if tgt in ranked[:k] else 0.0

def ndcg_at_k(ranked, tgt, k=10):
    for rank, iid in enumerate(ranked[:k], start=1):
        if iid == tgt:
            return 1.0 / np.log2(rank + 1)  # +1 in denom because ranks start at 1
    return 0.0

def eval_pool(df_eval: pd.DataFrame, scorer, k=10):
    """df_eval must have columns: uid, candidates (list[int]), target (int)."""
    hits, ndcgs = [], []
    for _, r in df_eval.iterrows():
        uid  = int(r["uid"])
        cnds = [int(x) for x in (r["candidates"] if isinstance(r["candidates"], (list, tuple, np.ndarray, pd.Series)) else [])]
        tgt  = int(r["target"])
        if not cnds:
            continue
        scores = scorer(uid, cnds)
        if scores.size == 0:
            continue
        order  = np.argsort(-scores)
        ranked = [cnds[i] for i in order]
        hits.append(hit_at_k(ranked, tgt, k))
        ndcgs.append(ndcg_at_k(ranked, tgt, k))
    return float(np.mean(hits)), float(np.mean(ndcgs))

# VAL (use for tuning)
hr_v, ndcg_v = eval_pool(val_eval, score_als, k=10)
print(f"[VAL]  ALS HR@10={hr_v:.3f}  NDCG@10={ndcg_v:.3f}")

# TEST (final report)
hr_t, ndcg_t = eval_pool(test_eval, score_als, k=10)
print(f"[TEST] ALS HR@10={hr_t:.3f}  NDCG@10={ndcg_t:.3f}")


[VAL]  ALS HR@10=0.290  NDCG@10=0.149
[TEST] ALS HR@10=0.180  NDCG@10=0.097


In [23]:
import numpy as np
import pandas as pd

def eval_precision_recall_f1(df_eval: pd.DataFrame, scorer, k=10):
    """df_eval: columns [uid, candidates(list[int]), target(int)]"""
    precisions, recalls, f1s = [], [], []
    for _, r in df_eval.iterrows():
        uid  = int(r["uid"])
        cnds = r["candidates"]
        if not isinstance(cnds, (list, tuple, np.ndarray, pd.Series)) or len(cnds) == 0:
            continue
        cnds = [int(x) for x in cnds]

        scores = scorer(uid, cnds)
        if scores.size == 0:
            continue

        # get top-k efficiently and sort them
        k_eff = min(k, scores.size)
        topk_idx = np.argpartition(-scores, k_eff-1)[:k_eff]
        topk_idx = topk_idx[np.argsort(-scores[topk_idx])]
        topk_items = [cnds[i] for i in topk_idx]

        hit = int(int(r["target"]) in topk_items)
        prec = hit / k                  # k is fixed per-user
        rec  = float(hit)               # 1 relevant item in LOO
        f1   = (2*prec*rec)/(prec+rec) if hit else 0.0

        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)

    return float(np.mean(precisions)), float(np.mean(recalls)), float(np.mean(f1s))

# Example usage with your ALS scorer:
p_v, r_v, f1_v = eval_precision_recall_f1(val_eval,  score_als, k=10)
p_t, r_t, f1_t = eval_precision_recall_f1(test_eval, score_als, k=10)
print(f"[VAL]  P@10={p_v:.4f}  R@10={r_v:.4f}  F1@10={f1_v:.4f}")
print(f"[TEST] P@10={p_t:.4f}  R@10={r_t:.4f}  F1@10={f1_t:.4f}")


[VAL]  P@10=0.0290  R@10=0.2897  F1@10=0.0527
[TEST] P@10=0.0180  R@10=0.1801  F1@10=0.0327


In [24]:
import numpy as np
import pandas as pd
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight

# ------------------------------------------------------
# Helper metric functions
# ------------------------------------------------------

def hit_at_k(ranked, tgt, k=10):
    return 1.0 if tgt in ranked[:k] else 0.0

def ndcg_at_k(ranked, tgt, k=10):
    for rank, iid in enumerate(ranked[:k], start=1):
        if iid == tgt:
            return 1.0 / np.log2(rank + 1)
    return 0.0

def precision_recall_f1(ranked, tgt, k=10):
    hit = int(tgt in ranked[:k])
    prec = hit / k
    rec = float(hit)         # because only 1 relevant item in LOO
    f1 = (2*prec*rec)/(prec+rec) if hit else 0.0
    return prec, rec, f1

def eval_pool(df_eval, scorer, k=10):
    hits, ndcgs, precs, recs, f1s = [], [], [], [], []
    for _, r in df_eval.iterrows():
        uid  = int(r["uid"])
        cnds = r["candidates"]
        if not isinstance(cnds, (list, tuple, np.ndarray, pd.Series)) or len(cnds) == 0:
            continue
        cnds = [int(x) for x in cnds]
        scores = scorer(uid, cnds)
        if scores.size == 0:
            continue
        order  = np.argsort(-scores)
        ranked = [cnds[i] for i in order]
        hits.append(hit_at_k(ranked, int(r["target"]), k))
        ndcgs.append(ndcg_at_k(ranked, int(r["target"]), k))
        p, r_, f1 = precision_recall_f1(ranked, int(r["target"]), k)
        precs.append(p); recs.append(r_); f1s.append(f1)
    return (
        float(np.mean(hits)), float(np.mean(ndcgs)),
        float(np.mean(precs)), float(np.mean(recs)), float(np.mean(f1s))
    )

# ------------------------------------------------------
# Training + evaluation function
# ------------------------------------------------------

def train_eval_als_once(seed, R, alpha, factors, reg, iters, use_gpu,
                        val_eval, test_eval, k=10):
    X = bm25_weight(R, K1=100, B=0.8).astype(np.float32)
    als = AlternatingLeastSquares(
        factors=factors,
        regularization=reg,
        iterations=iters,
        use_gpu=use_gpu,
        dtype=np.float32,
        random_state=seed,
    )
    als.fit(alpha * X)

    U_f = als.user_factors
    V_f = als.item_factors
    I   = V_f.shape[0]

    def score_als(uid, cand_iids):
        if not cand_iids: return np.asarray([])
        u = U_f[int(uid)]
        c = np.array([int(i) for i in cand_iids if 0 <= int(i) < I], dtype=np.int64)
        if c.size == 0: return np.asarray([])
        return V_f[c] @ u

    return eval_pool(val_eval,  score_als, k), eval_pool(test_eval, score_als, k)

# ------------------------------------------------------
# Grid search over hyperparameters
# ------------------------------------------------------

param_grid = [
    {"factors": 64,  "reg": 0.05, "alpha": 15, "iters": 150},
    {"factors": 96,  "reg": 0.05, "alpha": 15, "iters": 200},
    {"factors": 128, "reg": 0.1,  "alpha": 10, "iters": 200},
    {"factors": 128, "reg": 0.05, "alpha": 20, "iters": 200},
]

results = []
seeds = [0, 1, 2]  # average across a few random seeds

for params in param_grid:
    vals, tests = [], []
    for s in seeds:
        val_metrics, test_metrics = train_eval_als_once(
            seed=s,
            R=R,
            alpha=params["alpha"],
            factors=params["factors"],
            reg=params["reg"],
            iters=params["iters"],
            use_gpu=False,
            val_eval=val_eval,
            test_eval=test_eval,
            k=10,
        )
        vals.append(val_metrics)
        tests.append(test_metrics)

    vals, tests = np.array(vals), np.array(tests)
    val_mean, val_std = vals.mean(0), vals.std(0)
    test_mean, test_std = tests.mean(0), tests.std(0)

    results.append({
        **params,
        "val_HR": val_mean[0], "val_NDCG": val_mean[1],
        "val_P": val_mean[2],  "val_R": val_mean[3],  "val_F1": val_mean[4],
        "test_HR": test_mean[0], "test_NDCG": test_mean[1],
        "test_P": test_mean[2],  "test_R": test_mean[3], "test_F1": test_mean[4],
    })

# ------------------------------------------------------
# Results summary
# ------------------------------------------------------

res_df = pd.DataFrame(results)
print(res_df.sort_values("val_NDCG", ascending=False))


c:\Users\abdul\.venvs\ece1508gp\lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(
100%|██████████| 200/200 [00:06<00:00, 32.96it/s]
c:\Users\abdul\.venvs\ece1508gp\lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005388259887695312 seconds
  warnings.warn(
100%|██████████| 200/200 [00:06<00:00, 33.07it/s]
c:\Users\abdul\.venvs\ece1508gp\lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0011246204376220703 seconds
  warnings.warn(
100%|██████████| 200/200 [00:05<00:00, 36.97it/s]


   factors   reg  alpha  iters    val_HR  val_NDCG     val_P     val_R  \
0       64  0.05     15    150  0.324611  0.171206  0.032461  0.324611   
1       96  0.05     15    200  0.290966  0.149671  0.029097  0.290966   
2      128  0.10     10    200  0.256698  0.126655  0.025670  0.256698   
3      128  0.05     20    200  0.256698  0.126034  0.025670  0.256698   

     val_F1   test_HR  test_NDCG    test_P    test_R   test_F1  
0  0.059020  0.220152   0.113792  0.022015  0.220152  0.040028  
1  0.052903  0.189786   0.099426  0.018979  0.189786  0.034507  
2  0.046672  0.166322   0.085808  0.016632  0.166322  0.030240  
3  0.046672  0.167702   0.087037  0.016770  0.167702  0.030491  


### Best ALS hyperparameters: factors=64, regularization=0.05, alpha=15, iterations=150